### VIX Futures

A script for downloading VIX futures data from multiple expiries, 
and combining into a single pandas DataFrame.

In [13]:
import datetime
import numpy as np
import pandas as pd
import tqdm

import findatadownload as fdd

OUTPUT_FILENAME = '/Users/chris/OneDrive/programming/projects/finance/secdata/ts_data/vix_futures.csv'

In [2]:
def get_vix_expiries(start=None, end=None):
    """ Get all VIX expiry dates between the start/end. """
    start = None
    end = None

    if start is None:
        start = datetime.date(2013, 1, 1)

    if end is None:
        end = datetime.date.today() + pd.DateOffset(years=1)

    # Get the expiry dates from the meta data
    meta_filename = '/Users/chris/OneDrive/programming/projects/finance/secdata/meta_data/sec_info/FUT.csv'
    meta_df = pd.read_csv(meta_filename)
    
    # Only retrieve expiries that are between start/end
    expiries = meta_df.query('contract_code == "VX"').last_trade_date.astype(np.datetime64)
    expiries = [d.to_pydatetime().date() for d in expiries if start <= d and d <= end]    
    return expiries

In [ ]:
# Get all VIX expiries
expiries = get_vix_expiries()

# Get the index data
ind_data = fdd.download_time_series('cboe', 'VIX')
series_types = [s[s.find("(")+1:s.find(")")] for s in ind_data.columns]
ind_data.columns = series_types

df_list = [ind_data]
for expiry in tqdm.tqdm(expiries):
    # Form the ticker that is required by findatadownload
    base_ticker = 'VX{}'.format(expiry.strftime('%Y-%m-%d'))

    # Download the data from the .csv on the CBOE website
    df = fdd.download_time_series('cboe', base_ticker)

    # Remove the ticker name from the columns
    series_types = [s[s.find("(")+1:s.find(")")] for s in df.columns]
    df.columns = series_types
    
    # Add the expiry date to the columns
    df['expiration_date'] = expiry
    
    # Append the DataFrame to the list
    df_list.append(df)

In [26]:
# Save the data
df = pd.concat(df_list, axis=0)
#df.to_csv(OUTPUT_FILENAME)

In [48]:
# Load saved data

df = pd.read_csv(OUTPUT_FILENAME)
df.set_index('pricing_date', inplace=True)
df.index = pd.DatetimeIndex(fut_data.index)